# Artifact of Accessibility Issues in Ad-Driven Web Applications
Artifact of the paper "Accessibility Issues in Ad-Driven Web Applications" from ICSE 2025.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [1]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [2]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 46 compute_cascadelake_r nodes ready to use


### Reserve node

In [3]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start... This can take up to 60 seconds


Lease radhofanazizi_gmail_com-power-management has reached status active


### Create a server on the node

In [4]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server radhofanazizi_gmail_com-power-management's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,radhofanazizi_gmail_com-power-management
Id,3c6d20dc-78d5-4a4a-8ad1-2517de7f1c03
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.3.244 (v4) Type: fixed MAC: bc:97:e1:78:f1:a0
Network Name,sharednet1
Created At,2025-03-26T03:55:12Z
Keypair,trovi-6d35ec1
Reservation Id,efd49f45-f0cd-452a-a7b4-e7cc8f4febab
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


### Configure networking on the node

In [5]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.67 port 22.


Connection successful


## Run Experiment

Now, we can finally run the experiment. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [89]:
my_server.execute("rm -rf Web-Ads-Accessibility && git clone https://github.com/radhofan/Web-Ads-Accessibility.git")

Cloning into 'Web-Ads-Accessibility'...


<Result cmd='rm -rf Web-Ads-Accessibility && git clone https://github.com/radhofan/Web-Ads-Accessibility.git' exited=0>

In [104]:
my_server.execute("""
# 1. Clean existing installations (forceful)
sudo apt purge -y google-chrome* chromium* >/dev/null 2>&1
sudo rm -rf /usr/bin/google-chrome /usr/local/bin/chromedriver /opt/google/chrome >/dev/null 2>&1

# 2. Create necessary directories
sudo mkdir -p /usr/local/chrome114 /usr/local/chromedriver

# 3. Install Chrome 114 (with download verification)
if ! wget -q https://storage.googleapis.com/chrome-for-testing-public/114.0.5735.90/linux64/chrome-linux64.zip; then
    echo "Failed to download Chrome" >&2
    exit 1
fi
sudo unzip -o chrome-linux64.zip -d /usr/local/chrome114/
sudo ln -sf /usr/local/chrome114/chrome-linux64/chrome /usr/bin/google-chrome

# 4. Install ChromeDriver (with multiple fallbacks)
DRIVER_DOWNLOADED=0
for url in \
    "https://storage.googleapis.com/chrome-for-testing-public/114.0.5735.90/linux64/chromedriver-linux64.zip" \
    "https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip"
do
    if wget -q "$url"; then
        sudo unzip -o chromedriver*.zip -d /usr/local/chromedriver/
        DRIVER_DOWNLOADED=1
        break
    fi
done

if [ "$DRIVER_DOWNLOADED" -eq 0 ]; then
    echo "All ChromeDriver downloads failed!" >&2
    exit 1
fi

# 5. Setup ChromeDriver
sudo chmod +x /usr/local/chromedriver/chromedriver*
sudo ln -sf /usr/local/chromedriver/chromedriver* /usr/local/bin/chromedriver

# 6. Verify
echo "Chrome path: $(which google-chrome)"
echo "Chrome version: $(google-chrome --version)"
echo "ChromeDriver path: $(which chromedriver)"
chromedriver --version

# 7. Cleanup
rm -f chrome-linux64.zip chromedriver*.zip
""")

Archive:  chrome-linux64.zip
  inflating: /usr/local/chrome114/chrome-linux64/MEIPreload/manifest.json  
  inflating: /usr/local/chrome114/chrome-linux64/MEIPreload/preloaded_data.pb  
  inflating: /usr/local/chrome114/chrome-linux64/chrome  
  inflating: /usr/local/chrome114/chrome-linux64/chrome-wrapper  
  inflating: /usr/local/chrome114/chrome-linux64/chrome_100_percent.pak  
  inflating: /usr/local/chrome114/chrome-linux64/chrome_200_percent.pak  
  inflating: /usr/local/chrome114/chrome-linux64/chrome_crashpad_handler  
  inflating: /usr/local/chrome114/chrome-linux64/chrome_sandbox  
  inflating: /usr/local/chrome114/chrome-linux64/icudtl.dat  
  inflating: /usr/local/chrome114/chrome-linux64/libEGL.so  
  inflating: /usr/local/chrome114/chrome-linux64/libGLESv2.so  
  inflating: /usr/local/chrome114/chrome-linux64/libvk_swiftshader.so  
  inflating: /usr/local/chrome114/chrome-linux64/libvulkan.so.1  
  inflating: /usr/local/chrome114/chrome-linux64/nacl_helper  
  inflating: /

<Result cmd='\n# 1. Clean existing installations (forceful)\nsudo apt purge -y google-chrome* chromium* >/dev/null 2>&1\nsudo rm -rf /usr/bin/google-chrome /usr/local/bin/chromedriver /opt/google/chrome >/dev/null 2>&1\n\n# 2. Create necessary directories\nsudo mkdir -p /usr/local/chrome114 /usr/local/chromedriver\n\n# 3. Install Chrome 114 (with download verification)\nif ! wget -q https://storage.googleapis.com/chrome-for-testing-public/114.0.5735.90/linux64/chrome-linux64.zip; then\n    echo "Failed to download Chrome" >&2\n    exit 1\nfi\nsudo unzip -o chrome-linux64.zip -d /usr/local/chrome114/\nsudo ln -sf /usr/local/chrome114/chrome-linux64/chrome /usr/bin/google-chrome\n\n# 4. Install ChromeDriver (with multiple fallbacks)\nDRIVER_DOWNLOADED=0\nfor url in     "https://storage.googleapis.com/chrome-for-testing-public/114.0.5735.90/linux64/chromedriver-linux64.zip"     "https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip"\ndo\n    if wget -q "$url";

In [112]:
my_server.execute("""
curl -L -o IBM-Equal-Access-Accessibility-Checker.zip \
     https://github.com/IBMa/equal-access/archive/refs/tags/4.0.2.zip
""")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  100M    0  100M    0     0  8738k      0 --:--:--  0:00:06 --:--:-- 7915k.5M      0 --:--:--  0:00:01 --:--:-- 22.0M8M 0 38.4M    0     0  10.4M      0 --:--:--  0:00:03 --:--:-- 12.6MM 0  9624k      0 --:--:--  0:00:05 --:--:-- 10.5M0k  0 68.3M    0     0  9156k      0 --:--:--  0:00:07 --:--:-- 7815k0  8917k      0 --:--:--  0:00:08 --:--:-- 7595k    0 83.0M    0     0  8829k      0 --:--:--  0:00:09 --:--:-- 7551k    0 89.8M    0     0  8641k      0 --:--:--  0:00:10 --:--:-- 7525k   0 96.9M    0     0  8529k      0 --:--:--  0:00:11 --:--:-- 7386k--  0:00:11 --:--:-- 7964k


<Result cmd='\ncurl -L -o IBM-Equal-Access-Accessibility-Checker.zip      https://github.com/IBMa/equal-access/archive/refs/tags/4.0.2.zip\n' exited=0>

In [114]:
my_server.execute("chmod +x Web-Ads-Accessibility/reproduce.sh")
my_server.execute("bash Web-Ads-Accessibility/reproduce.sh")

ERROR: File or directory already exists: '/home/cc/miniconda'
If you want to update an existing installation, use the -u option.


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.

Running `shell init`, which:
 - modifies RC file: "/home/cc/.bashrc"
 - generates config for root prefix: "/home/cc/miniconda"
 - sets mamba executable to: "/home/cc/miniconda/bin/mamba"
The following has been added in your "/home/cc/.bashrc" file

# >>> mamba initialize >>>
# !! Contents within this block are managed by 'mamba shell init' !!
export MAMBA_EXE='/home/cc/miniconda/bin/mamba';
export MAMBA_ROOT_PREFIX='/home/cc/miniconda';
__mamba_setup="$("$MAMBA_EXE" shell hook --shell bash --root-prefix "$MAMBA_ROOT_PREFIX" 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__mamba_setup"
else
    alias mamba="$MAMBA_EXE"  # Fallback on help from mamba activate
fi
unset __mamba_setup
# <<< mamba initialize <<<

Found conda-prefix at '/home/cc/miniconda/envs/icse'. Overwrite?: [y/N] Aborted.


critical libmamba Aborting.


mkdir: cannot create directory ‘Web-Ads-Accessibility/src/output’: File exists
mkdir: cannot create directory ‘Web-Ads-Accessibility/src/differences’: File exists
Traceback (most recent call last):
  File "/home/cc/Web-Ads-Accessibility/src/sele.py", line 124, in <module>
    crawl_ibm_extension(website, adblock=False)
  File "/home/cc/Web-Ads-Accessibility/src/sele.py", line 52, in crawl_ibm_extension
    driver = webdriver.Chrome(service=service, options=chrome_options)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/selenium/webdriver/chrome/webdriver.py", line 45, in __init__
    super().__init__(
  File "/home/cc/miniconda/lib/python3.12/site-packages/selenium/webdriver/chromium/webdriver.py", line 66, in __init__
    super().__init__(command_executor=executor, options=options)
  File "/home/cc/miniconda/lib/python3.12/site-packages/selenium/webdriver/remote/webdriver.py", line 250, in __init__
    self

Crawling alaskanewssource.com with ads
alaskanewssource.com crashed
Error details:


...

----------------------Figure 8b--------------------


Traceback (most recent call last):
  File "/home/cc/Web-Ads-Accessibility/src/reproducible/fig8b.py", line 10, in <module>
    normal_scores = pd.read_csv('normal_8b.csv', header=None).squeeze()
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 1880, in _make_engine

----------------------Figure 8c--------------------


  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 1880, in _make_engine
    self.handles = get_handle(
                   ^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/common.py", line 873, in get_handle
    handle = open(
             ^^^^^
FileNotFoundError: [Errno 2] No such file or directory

UnexpectedExit: Encountered a bad command exit code!

Command: 'bash Web-Ads-Accessibility/reproduce.sh'

Exit code: 1

Stdout: already printed

Stderr: already printed

